In [19]:
import pandas as pd
import numpy as np
import kaleido
import plotly.express as px
import plotly.io as pio
import plotly.graph_objects as go
import ipywidgets as widgets
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, StandardScaler, LabelEncoder
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from plotly.subplots import make_subplots
from sklearn.preprocessing import  OneHotEncoder, StandardScaler
from sklearn.neighbors import NearestNeighbors
from sklearn.compose import ColumnTransformer
from sklearn.cluster import KMeans, DBSCAN
from sklearn.metrics import  silhouette_score
from IPython.display import display, clear_output


In [20]:
df = pd.read_csv('uber_data_sample.csv')

## LE MODELE DBSCAN
Nous utiliserons DBSCAN avec les mêmes éléments de KMEANS.
Ce modèle nécessite 2 paramètres : epsilone et min_sample
En gros c'est à nous de décider combien doit il avoir de point dans un perimètre donné pour que ces points intègre le cluster.
Ce systeme va permettre d'écarter rapidement et assez naturellement les outliers.
Néanmoins pour nous aider à définir au mieux l'epsilone nous utiliserons NearestNeighbors que nous verons plus bas

In [21]:
day = 'Thursday'
hours = 17

## DBSCAN JOUR

In [22]:
# On filtre le DataFrame pour ne garder que le jour sélectionné.
# Pas de preprocessing nécessaire car le clustering DBSCAN utilisera uniquement les colonnes 'Lat' et 'Lon'.
mask_day = df['dayname'] == day 
df_day = df[mask_day]
df_day1 = df_day[['Lat','Lon']]


Comme cité plus haut nous allons utiliser NearestNeighbors. 
C'est une classe de scikit-learn qui va nous aider à analyser les distances entre les points d’un dataset. Elle permet notamment de :

Trouver les points les plus proches de chaque point dans le dataset.

Mesurer ces distances pour détecter des zones plus ou moins denses et repérer les points isolés.

Préparer des paramètres pour DBSCAN, notamment la valeur de eps qui définit le rayon autour d’un point pour créer un cluster.

In [23]:
# Nous prendrons les 20 plus proches voisins
neigh = NearestNeighbors(n_neighbors=20)
nbrs = neigh.fit(df_day1)
distances, indices = nbrs.kneighbors(df_day1)
# Trier les distances dans l'ordre croissant
distances = np.sort(distances[:, -1])
# Graphique pour trouver le "coude"
df_dist = pd.DataFrame({
    "Points": range(len(distances)),
    "Distance": distances
})

fig = px.line(df_dist, x="Points", y="Distance",
              title="Graphique epsilon",
              labels={"Points": "Points", "Distance": "Distance au 20ème plus proche voisin"})
fig.show()

Le graphique montre une forte augmentation à partir de 0,05, qui correspond au “coude”. Cependant, nous n’utiliserons pas cette valeur car elle produirait trop peu de clusters. Nous choisirons plutôt un epsilon de 0,01, ce qui permettra d’obtenir des clusters plus fins et mieux répartis sur la zone étudiée. 
Dans notre visualisation nous écarterons les outliers identifié par le cluster -1

In [24]:
db1 = DBSCAN(eps=0.01, min_samples=20, metric="manhattan")
df_day1 = df_day1.copy()
df_day1.loc[:, "cluster"] = db1.fit_predict(df_day1)

fig = px.scatter_map(
        df_day1[df_day1['cluster'] != -1],
        lat="Lat",
        lon="Lon",
        color="cluster",
        map_style="carto-positron",
        zoom=9,
        title= f"HOT POINT {day}" 
)

fig.show()

fig2 = px.scatter_map(
        df_day1,
        lat="Lat",
        lon="Lon",
        color="cluster",
        map_style="carto-positron",
        title= f"HOT POINT {day} - Avec OUTLIERS"  
)

fig2.show()

Cette première carte montre 7 clusters, dont un très dense regroupant plus de 6300 points sur un total de 7483.
La zone extrêmement dense est difficile à découper correctement avec notre modèle, ce qui peut limiter la granularité de l’analyse dans cette région.

### DBSCAN HEURE

In [25]:
# On filtre le DataFrame pour ne garder que l'heure sélectionnée.
# Pas de preprocessing nécessaire car le clustering DBSCAN utilisera uniquement les colonnes 'Lat' et 'Lon'.
mask_hour = df['hour'] == hours
df_hour = df[mask_hour]
df_hour1= df_hour[['Lat', 'Lon']]


In [26]:
# Nous prendrons les 20 plus proches voisins
neigh = NearestNeighbors(n_neighbors=20)
nbrs = neigh.fit(df_hour1)
distances, indices = nbrs.kneighbors(df_hour1)
# Trier les distances dans l'ordre croissant
distances = np.sort(distances[:, -1])
# Graphique pour trouver le "coude"
df_dist = pd.DataFrame({
    "Points": range(len(distances)),
    "Distance": distances
})

fig = px.line(df_dist, x="Points", y="Distance",
              title="Graphique epsilon",
              labels={"Points": "Points", "Distance": "Distance au 20ème plus proche voisin"})
fig.show()

Notre courbe montre un coude qui se dessine autour de 0.05.
Cependant, comme dans notre analyse précédente, nous retiendrons la valeur de 0.01 afin d’obtenir une meilleure granularité des clusters.

In [27]:

db = DBSCAN(eps=0.01, min_samples=20, metric="manhattan") 
df_hour1 = df_hour1.copy()
df_hour1.loc[:, "cluster"] = db.fit_predict(df_hour1)

fig = px.scatter_map(
        df_hour1[df_hour1['cluster'] != -1],
        lat="Lat",
        lon="Lon",
        color="cluster",
        map_style="carto-positron",
        zoom=9,
        title= f"HOT POINT {hours} H"  
)

fig.show()

fig2 = px.scatter_map(
        df_hour1,
        lat="Lat",
        lon="Lon",
        color="cluster",
        map_style="carto-positron",
        title= f"HOT POINT {hours} H - Avec OUTLIERS"  
)

fig2.show()

Notre carte met en évidence 7 clusters, dont un particulièrement dense (2690 observations sur 3362). Comme lors de notre analyse précédente, le modèle rencontre des difficultés à segmenter cette zone à forte concentration.

### DBSCAN JOUR HEURE

In [28]:
# On filtre le DataFrame pour ne garder que l'heure et le jour sélectionné.
# Pas de preprocessing nécessaire car le clustering DBSCAN utilisera uniquement les colonnes 'Lat' et 'Lon'.
mask_day_hour = (df['dayname'] == day) & (df['hour'] == hours)
day_dh = df[mask_day_hour]
day_dh1 = day_dh[['Lat','Lon']]


In [29]:
# Nous prendrons les 20 plus proches voisins
neigh = NearestNeighbors(n_neighbors=20)
nbrs = neigh.fit(day_dh1)
distances, indices = nbrs.kneighbors(day_dh1)
# Trier les distances dans l'ordre croissant
distances = np.sort(distances[:, -1])
# Graphique pour trouver le "coude"
df_dist = pd.DataFrame({
    "Points": range(len(distances)),
    "Distance": distances
})

fig = px.line(df_dist, x="Points", y="Distance",
              title="Graphique epsilon",
              labels={"Points": "Points", "Distance": "Distance au 20ème plus proche voisin"})
fig.show()

La courbe suggère un "coude" autour de 0.05 pour epsilon. 
Cependant, afin d'obtenir une granularité plus fine et des clusters plus petits tout en restant significatifs,nous choisissons une valeur d'epsilon plus faible : 0.007.

In [30]:
db = DBSCAN(eps=0.007, min_samples=20, metric="manhattan")
day_dh1 = day_dh1.copy()                
day_dh1.loc[:, "cluster"] = db.fit_predict(day_dh1)

fig1 = px.scatter_map(
        day_dh1[day_dh1['cluster'] != -1],
        lat="Lat",
        lon="Lon",
        color="cluster",
        map_style="carto-positron",
        zoom=10,
        title= f"HOT POINT {day} {hours} H"  
)

fig1.show()

fig2 = px.scatter_map(
        day_dh1,
        lat="Lat",
        lon="Lon",
        color="cluster",
        map_style="carto-positron",
        zoom=10,
        title= f"HOT POINT {day} {hours} H - Avec OUTLIERS"  
)

fig2.show()

Le modèle identifie 3 clusters : un cluster très dense (regroupant un peu moins de 50 % des points) et deux clusters de taille plus réduite. L’ajustement des paramètres a permis d’affiner la granularité et de mieux segmenter la zone particulièrement dense mise en évidence lors de nos analyses précédentes. Nous examinons à présent le comportement du modèle sur les autres jours de la semaine et à différentes heures de la journée.

In [31]:
jours_ordre = ["Monday","Tuesday","Wednesday","Thursday","Friday","Saturday","Sunday"]
all_hours = list(range(24))

def create_figure_dbscan(days, hours, eps=0.007, min_samples=20, metric='manhattan'):
    # Filtre jour + heure
    mask_day_hour = (df['dayname'] == days) & (df['hour'] == hours)
    df_plot = df[mask_day_hour].copy()  

    # Coordonnées uniquement
    coords = df_plot[["Lat", "Lon"]].values

    # DBSCAN
    dbscan = DBSCAN(eps=eps, min_samples=min_samples, metric=metric)
    df_plot.loc[:, "DBSCAN"] = dbscan.fit_predict(coords)
    clustered = df_plot[df_plot["DBSCAN"] != -1]

    if clustered.empty:
        # Cas où il y a des données mais aucun cluster trouvé
        fig = px.scatter_map(
            pd.DataFrame({"Lat": [], "Lon": [], "DBSCAN": []}),
            lat="Lat",
            lon="Lon",
            color="DBSCAN",
            zoom=10,
            map_style="carto-positron",
            width=1000,
            height=700,
            title=f"Clusters DBSCAN - {days} {hours}h (aucun cluster trouvé)"
        )
    else:
        # Cas normal avec clusters
        fig = px.scatter_map(
            clustered,
            lat="Lat",
            lon="Lon",
            color="DBSCAN",
            hover_name="DBSCAN",
            zoom=10,
            map_style="carto-positron",
            width=1000,
            height=700,
            title=f"Clusters DBSCAN - {days} {hours}h"
        )
    return fig

# Widgets Dropdown pour choisir jour + heure
dropdown_day = widgets.Dropdown(
    options=jours_ordre,
    value="Thursday",
    description='Jour:',
)

dropdown_hour = widgets.Dropdown(
    options=all_hours,
    value=12,
    description='Heure:',
)

output = widgets.Output()

def update_plot(_):
    with output:
        clear_output(wait=True)
        fig = create_figure_dbscan(dropdown_day.value, dropdown_hour.value)
        display(fig)

# Observer les 2 menus
dropdown_day.observe(update_plot, names="value")
dropdown_hour.observe(update_plot, names="value")

# Affichage initial
with output:
    fig = create_figure_dbscan(dropdown_day.value, dropdown_hour.value)
    display(fig)

display(dropdown_day, dropdown_hour, output)


Dropdown(description='Jour:', index=3, options=('Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Satur…

Dropdown(description='Heure:', index=12, options=(0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17…

Output()

DBSCAN n’arrive pas à identifier des clusters à chaque heure de la journée pour tous les jours. En effet, le paramètre eps que nous avons choisi se base sur la période la plus dense afin de détecter les clusters significatifs.

Pour les heures ou les jours avec moins de trafic, le modèle ne trouve pas de clusters pertinents, ce qui génère beaucoup d’outliers. Comme nous avons filtré ces outliers, certaines journées ne montrent des clusters que pour quelques heures, correspondant aux périodes où DBSCAN a pu détecter des regroupements.

Le modèle ne généralise donc pas bien pour des périodes de faible densité. Pour remédier à cela, il serait intéressant de segmenter la journée en 3 ou 4 périodes (matin, après-midi, soir, nuit) et de déterminer un eps adapté à chaque période, à appliquer lors du choix de l’heure.


### Pour conclure

DBSCAN présente plusieurs avantages dans notre cas :

- Il permet de détecter des clusters denses de manière efficace, même en présence de points isolés.

- Il identifie automatiquement les regroupements sans avoir besoin de spécifier le nombre de clusters à l’avance.

- Il écarte naturellement les outliers, ce qui facilite la lecture des zones d’activité.

Cependant, il présente aussi certaines limites :

- Son paramétrage (eps et min_samples) influence fortement la détection des clusters et doit être adapté à la densité des points.

- Pour des périodes ou zones avec peu de trafic, le modèle peine à identifier des clusters, générant ainsi beaucoup d’outliers.

- La variation de densité au cours de la journée peut limiter la continuité des clusters sur l’ensemble des heures.

### Recommandation pour l’utilisation dans le cas Uber

* KMeans reste le choix le plus adapté pour notre cas :

Il détecte toujours des “hotpoints” même lorsque la densité de points est faible, ce qui permet d’avoir une couverture complète sur toutes les heures et tous les jours.

Les centroïdes identifiés sont faciles à interpréter et permettent de visualiser clairement les zones d’activité principales.

Sa simplicité et sa rapidité en font un outil robuste pour traiter un gros volume de trajets Uber.

* DBSCAN, bien qu’efficace pour identifier des clusters denses et filtrer les outliers :

Ne parvient pas à détecter des clusters lorsque la densité de points est faible, ce qui laisse des heures “vides” dans la frise temporelle.

Nécessite un paramétrage très précis (eps, min_samples) qui peut varier selon la période de la journée.

Sa sensibilité à la densité rend l’interprétation moins constante sur l’ensemble des données.

#### Conclusion pratique :
Pour une application Uber où l’objectif est de repérer des zones d’activité sur toute la journée et sur tous les jours, KMeans est recommandé. Il fournit des résultats stables et interprétables, même dans les périodes de faible trafic, ce qui est essentiel pour la visualisation des “hotpoints”. DBSCAN peut être envisagé comme analyse complémentaire si l’on souhaite identifier uniquement les clusters très denses et filtrer les points isolés.